# Tidal data exploration

## Analysis of data from NOAA tidal stations

### Guiding Questions:
    
**How many places in the US have large enough tidal ranges to harvest tidal energy economically?**
  - *large enough = avg. tidal range > 10 ft according to [U.S. Energy Information Administration](https://www.eia.gov/energyexplained/hydropower/tidal-power.php#:~:text=Today%2C%20there%20are%20tidal%20energy,of%20at%20least%2010%20feet.)
  
**How many places in the US have a tidal range >= 18ft**
  - *Two of the largest and most economical tidal power projects in the world, [Sihwa tidal power station in South Korea](https://en.wikipedia.org/wiki/Sihwa_Lake_Tidal_Power_Station#:~:text=This%20slightly%20unconventional%20and%20relatively,7.8%20m%20(26%20ft).) and [La Rance tidal barrage in France](https://tethys.pnnl.gov/project-sites/la-rance-tidal-barrage#:~:text=Location-,The%20facility%20is%20located%20on%20the%20estuary%20of%20the%20Rance,of%209%2C600m3%2Fs.) have average tidal ranges >= 18ft

### Objectives:
- **determine average tidal range for each station that has tidal data**
- **identify stations with an avg. tidal range >= 10ft, and an avg. tidal range >= 18ft**
- **create a new CSV file with stations and their IDs for further exploration**


In [77]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import all station ids from a csv that contains all ids in a single column 
all_stations_2d = pd.read_csv('../csvs/all_station_ids.csv').values.tolist()
all_stations = [i[0] for i in all_stations_2d]

# create empty lists for highest highs, lowest lows, and tidal ranges of each station
avg_t_ranges = []
cleansed_station_dicts = []
gte_10ft_lt_18ft_dicts = []
gte_18ft_dicts = []
path = "../csvs/tide_predictions/"

# counter variable and list to hold # of stations that should be included on x-axis for plots
ex = 0
exes = []

# find average high and low for each station to calc tidal range
for station in all_stations:
    index = all_stations.index(station)
    file_name = path + str(all_stations[index]) + "_2020.csv"
    df = pd.read_csv(file_name)
    
    if df.water_level.empty == False:
        all_highs = df.loc[df['type'] == 'H']["water_level"].tolist()
        all_lows = df.loc[df['type'] == 'L']["water_level"].tolist()
        avg_high, avg_low = np.mean(all_highs), np.mean(all_lows)
        avg_tidal_range = abs(avg_high - avg_low).round(1)
        station_dict = {'station_id': all_stations[index], 'avg_tidal_range': avg_tidal_range, 'range_gte_10ft': avg_tidal_range >= 10.0, 'range_gte_18ft': avg_tidal_range >= 18.0}
        
        if avg_tidal_range >= 10.0:
            cleansed_station_dicts.append(station_dict)
            if avg_tidal_range < 18.0:
                gte_10ft_lt_18ft_dicts.append(station_dict)
            if avg_tidal_range >= 18.0:
                gte_18ft_dicts.append(station_dict)

print(f"Tidal stations in U.S. with avg. tidal range >= 10f < 18ft: {len(gte_10ft_lt_18ft_dicts)}")
print(f"Tidal stations in U.S. with avg. tidal range >= 18ft: {len(gte_18ft_dicts)}")


Tidal stations in U.S. with avg. tidal range >= 10f < 18ft: 70
Tidal stations in U.S. with avg. tidal range >= 18ft: 8


In [86]:
cleansed_stations_df = pd.DataFrame(cleansed_station_dicts)

csvs_path = '../csvs/'

cleansed_stations_df.to_csv(csvs_path+'stations_with_avg_tidal_range_gte_10ft.csv',index=False)